In [23]:
import MMAl
from MMAl import *

In [24]:
from EnergyFunctions import *
from DataProcessing import *
from DataImporting import *
import random
import math
import time
import datetime as dt

Delta_List = []; energy_list = []
# @profile
def main(iterations, temp, exponent, energy_if_false, peak_list, residue_list, create_chain = False):
    start_time = time.time()  # notes start time

    print('iterations:', iterations, 'temp:', temp, 'exponent:', exponent, 'eif:', energy_if_false)  # prints initial
    if should_save_to_file:
        fh.write('iterations:{}, temp:{}, exponent:{}, eif:{}\n'.format(iterations, temp, exponent, energy_if_false))

    index_list = list(range(len(peak_list)))  # creates index list to link residue and peak lists
    assert len(index_list) == len(residue_list)
    random.shuffle(index_list, lambda: random.random())  # shuffles index list
    og_index_list = index_list  # stores original index list
    count_temp = 0  # count temp is reset
    count = 0  # counts iterations

    # iterations
    for i in range(iterations):
        count += 1
        count_temp += 1
        a = random.randint(0, len(index_list) - 1)  # select a random place from the index list
        new_index_list = index_list[:]  # makes a deep copy

        if create_chain:
            r, f = chain_creator(index_list, peak_list, a)  # gets how big the chain should be

            if True:
                a_list = [x for x in range(a + r, a + f + 1)]  # all 'a' we're switching
                while True:
                    b = random.randint(-1 * r, len(index_list) - 1 - f)  # returns 'b' inclusive
                    b_list = [x for x in range(b + r, b + f + 1)]
                    if set(a_list).isdisjoint(b_list):  # Return True if two sets have a null intersection.
                        break
                new_index_list[a + r: a + f + 1], new_index_list[b + r: b + f + 1], = \
                    new_index_list[b + r: b + f + 1], new_index_list[a + r: a + f + 1]  # swaps

        else:
            b = random.randint(0, len(index_list) - 1)
            new_index_list[a], new_index_list[b] = new_index_list[b], new_index_list[a]
            a_list = [a]; b_list = [b]

        if swap_evaluator(a_list, b_list, index_list, new_index_list, temp,
                          peak_list, residue_list, energy_if_false, Delta_List):
            index_list = new_index_list

        if temp == 0:
            print('hi')

        # TODO: FIX THIS SO IT DOESNT TAKE UP TIME
        elif count_temp == int(iterations*((math.log(1/temp, exponent))**-1)):
            count_temp = 0
            temp = temp * exponent

        if dt.datetime.now() > date: break

    # Data for runtime
    if True:
        end_time = time.time()
        print('time taken (sec): {}'.format(end_time - start_time))
        if should_save_to_file: fh.write('time taken (sec): {}\n'.format(end_time - start_time))

        print('number of iterations: {}'.format(count))
        if should_save_to_file: fh.write('number of iterations: {}\n'.format(count))

        print('final temperature: {}'.format(temp))
        if should_save_to_file: fh.write('final temperature: {}'.format(temp))

        print('og index list:', [x for x in og_index_list])
        if should_save_to_file: fh.write('og index list: {}\n'.format([x for x in og_index_list]))

        og_energy = eval_energy(og_index_list, peak_list, residue_list, energy_if_false)[0]
        print('og index list energy:', og_energy)
        if should_save_to_file: fh.write('og index list energy: {}\n'.format(og_energy))

        print('index list:', [x for x in index_list])
        if should_save_to_file: fh.write('index list: {}\n'.format([x for x in index_list]))

        energy = eval_energy(index_list, peak_list, residue_list, energy_if_false)[0]
        energy_list.append(energy)
        print('index list energy:', energy)
        if should_save_to_file: fh.write('index list energy: {}'.format(energy))

    # RETURNS HERE
    return og_index_list, og_energy, index_list, energy

# @profile
def swap_evaluator(a_list, b_list, index_list, new_index_list, temp, peak_list, residue_list, energy_if_false, Delta_List):
    global delta
    delta = main_energy_function(a_list, b_list, index_list, new_index_list, peak_list, residue_list, energy_if_false, Delta_List)
    if should_append_DL:
        Delta_List.append([delta, a_list, b_list, [index_list[x] for x in a_list], [index_list[x] for x in b_list], index_list, new_index_list])

    if delta <= 0:
        return True
    elif temp == 0:
        return False
    elif delta > 0:
        p = math.exp(-delta/temp)
        if random.random() <= p:
            return True
        else:
            return False

# @profile
def chain_creator(index_list, peak_list, a):
    # 120 is probably a good cutoff
    f = 0  # forward
    r = 0  # reverse
    if True:
        while True:
            i = a + f  # recall, a is the random position chosen on the index list, so i is the most forward index
            if i >= len(index_list) - 1:  # ensures chain doesn't extend beyond index list
                break
            ca_ca_prime_diff = CA_CA_prime_diff(i, index_list[i], index_list, peak_list, [])  # gets energy for next pk
            cb_cb_prime_diff = CB_CB_prime_diff(i, index_list[i], index_list, peak_list, [])
            sum_energy = ca_ca_prime_diff + cb_cb_prime_diff
            if sum_energy < 600:
                p = math.exp(-0.007 * sum_energy)
                if random.random() < p:
                    f += 1
                else:
                    break
            else:
                break

        while True:
            i = a + r - 1
            if i < 0:
                break
            ca_ca_prime_diff = CA_CA_prime_diff(i, index_list[i], index_list, peak_list, [])
            cb_cb_prime_diff = CB_CB_prime_diff(i, index_list[i], index_list, peak_list, [])
            sum_energy = ca_ca_prime_diff + cb_cb_prime_diff
            if sum_energy < 600:
                p = math.exp(-0.007 * sum_energy)
                if random.random() < p:
                    r -= 1
                else:
                    break
            else:
                break

    assert r <= 0; assert f >= 0
    return r, f


# HOW LONG DOES THE PROGRAM RUN??? (you can also add a specific time instead of a delta)
date = dt.datetime.now() + dt.timedelta(days=5, hours=11, minutes=58)
if __name__ == '__main__':
    if dt.datetime.now() > date:
        raise Exception('end time before begin time')

    should_save_to_file = True  # SHOULD IT WRITE TO A FILE
    if should_save_to_file:
        file_name = str(dt.datetime.now().isoformat())
        fh = open('mmals/' + file_name + '.txt', 'w+')

    # starts program
    if True:
        time_start = dt.datetime.now()
        print('program started at:{}'.format(time_start))
        peak_list, residue_list = main_data_processing()
        for i in range(10):  # HOW MANY TRIALS SHOULD HAPPEN
            result = main(int(iterations), temp, exponent, energy_if_false, peak_list, residue_list, create_chain=True)

            print('Done\n\n')
            if should_save_to_file:
                fh.write('Done\n\n')

            if dt.datetime.now() > date:
                break
        time_finish = dt.datetime.now()
        print('program ended at: {} \n time delta: {}'.format(time_finish, time_finish - time_start))

    if should_save_to_file:
        fh.close()


program started at:2020-12-22 13:47:04.125735
iterations: 1001 temp: 1000000.0 exponent: 0.9 eif: 300.0
time taken (sec): 2.362197160720825
number of iterations: 1001
final temperature: 129.0070078170105
og index list: [439, 181, 214, 474, 384, 265, 83, 56, 9, 426, 183, 454, 300, 11, 414, 409, 301, 462, 264, 313, 394, 298, 149, 295, 435, 338, 209, 376, 193, 64, 359, 249, 155, 416, 205, 234, 332, 401, 74, 315, 382, 355, 170, 238, 62, 27, 106, 131, 157, 236, 284, 311, 167, 73, 94, 26, 4, 237, 189, 309, 200, 134, 68, 194, 150, 82, 360, 34, 340, 128, 140, 154, 377, 357, 57, 438, 417, 457, 319, 305, 132, 117, 334, 271, 270, 424, 302, 460, 252, 318, 137, 39, 116, 294, 297, 388, 375, 446, 50, 254, 354, 344, 196, 320, 304, 373, 187, 469, 356, 118, 229, 25, 0, 147, 144, 67, 350, 124, 6, 90, 24, 2, 349, 179, 248, 422, 258, 451, 312, 427, 421, 163, 473, 110, 177, 164, 28, 1, 202, 35, 433, 292, 299, 239, 296, 76, 442, 130, 115, 465, 331, 268, 286, 77, 201, 266, 379, 36, 41, 38, 429, 148, 393, 211,

In [25]:
total = int()
good = int()
for peak in peak_list:
    TROSYHShift = peak.get_data('TROSYHShift')
    nearbyHShift = peak.get_data('NearbyHShift')
    if len(nearbyHShift) > 0:
        delta_list = [abs(x - TROSYHShift) for x in nearbyHShift]
        min_value = min(delta_list)
        total += 1
        if min_value > 0.05:
            print(peak.peak_properties)
            print('TROSYHShift: {}'.format(TROSYHShift))
            print('List of NOESY Shifts: {}'.format(nearbyHShift))
            print('Noesy shift minus Trosy H shift: {}'.format([round(x, 3) for x in delta_list]))
            print('\n')
        else:
            good += 1
    else:
        continue
print('Of the {} peaks with NOESY data, {} had cross peaks within 0.05 ppm ({}%)'.format(total, good, good/total))

{'peakNumber': 0, 'TROSYHShift': 2.9610000000000003, 'TROSYNShift': 108.23, 'CAShift': 43.33, 'CAPrimeShift': 51.69, 'CBShift': None, 'CBPrimeShift': 16.52, 'COPrimeShift': 177.7, 'NOESYHShift': [2.971, 2.97, 2.97, 2.97, 2.97], 'NOESYNShift': [108.267, 108.278, 108.243, 108.273, 108.269], 'NearbyHShift': [8.496, 7.63, 7.875, 10.944, 9.763]}
TROSYHShift: 2.9610000000000003
List of NOESY Shifts: [8.496, 7.63, 7.875, 10.944, 9.763]
Noesy shift minus Trosy H shift: [5.535, 4.669, 4.914, 7.983, 6.802]


{'peakNumber': 13, 'TROSYHShift': 7.279, 'TROSYNShift': 122.93, 'CAShift': 56.72, 'CAPrimeShift': 55.48, 'CBShift': 63.29, 'CBPrimeShift': 33.53, 'COPrimeShift': 172.5, 'NOESYHShift': [7.209, 7.211], 'NOESYNShift': [122.856, 122.892], 'NearbyHShift': [8.849, 7.201]}
TROSYHShift: 7.279
List of NOESY Shifts: [8.849, 7.201]
Noesy shift minus Trosy H shift: [1.57, 0.078]


{'peakNumber': 107, 'TROSYHShift': 8.962, 'TROSYNShift': 120.53, 'CAShift': 60.47, 'CAPrimeShift': 61.806, 'CBShift': 30.66,

In [30]:
for residue in residue_list:
    print(residue.residue_properties, '\n')
    assert 1 not in residue.get_data('closebyAminoAcids')


{'residue_name': 'S', 'index': 1, 'secondaryStructure': None, 'CAExpected': 58.35, 'CAExpectedSD': 1.78, 'CBExpected': 63.88, 'CBExpectedSD': 1.24, 'closebyAminoAcids': [], 'closebyAminoAcidsDist': [], 'secondary_structure': 'R'} 

{'residue_name': 'T', 'index': 2, 'secondaryStructure': None, 'CAExpected': 61.59, 'CAExpectedSD': 2.04, 'CBExpected': 69.75, 'CBExpectedSD': 1.29, 'closebyAminoAcids': [4, 5, 7, 8, 9, 11, 12, 255, 256, 257], 'closebyAminoAcidsDist': [4.99, 4.61, 7.79, 5.12, 6.32, 7.69, 7.03, 6.67, 3.54, 6.63], 'secondary_structure': 'R'} 

{'residue_name': 'P', 'index': 3, 'secondaryStructure': None, 'CAExpected': 63.53, 'CAExpectedSD': 1.26, 'CBExpected': 31.87, 'CBExpectedSD': 0.96, 'closebyAminoAcids': [], 'closebyAminoAcidsDist': [], 'secondary_structure': 'R'} 

{'residue_name': 'K', 'index': 4, 'secondaryStructure': None, 'CAExpected': 56.4, 'CAExpectedSD': 1.8, 'CBExpected': 32.57, 'CBExpectedSD': 1.3, 'closebyAminoAcids': [2, 5, 6, 8, 9, 234, 235, 236, 256], 'closeb

In [17]:
secondary_structure(residue_list)

